# BYOC training for paddleOCR

## step1: upload data

In [22]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role

In [23]:
sess = sage.Session()

WORK_DIRECTORY = "./input/data"

# S3 prefix
prefix = "DEMO-paddle-byo"

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)
print(data_location)

s3://sagemaker-eu-west-1-707684582322/DEMO-paddle-byo


In [24]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

PROJECT_ID = "sagemaker-p-5an0os9jqfdi"

## Create training job for training

In [26]:
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{PROJECT_ID}-training-imagebuild:latest'
print('Training image location: ',image)


Training image location:  707684582322.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-p-5an0os9jqfdi-training-imagebuild:latest


In [27]:
sess = sage.Session()
hyperparameters = {"epoch_num": 10,
                  "print_batch_step":5,
                  "save_epoch_step":3,
                  'pretrained_model':'/opt/program/pretrain/ch_ppocr_mobile_v2.0_rec_train/best_accuracy'}

train = sage.estimator.Estimator(
    image,
    role,
    instance_count = 1,
    sagemaker_session=sess,
    instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters,
)


train.fit(data_location, wait=False)